In [ ]:
from google.colab import files
uploaded = files.upload()

Saving RoomieMatch.db to RoomieMatch (1).db


In [ ]:
import shutil
shutil.move("RoomieMatch.db", "/content/RoomieMatch.db")

'/content/RoomieMatch.db'

In [ ]:
import sqlite3

In [ ]:
# Connect to the database
connection = sqlite3.connect("/content/RoomieMatch.db")

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('sqlite_sequence',), ('user_hobbies',), ('hobbies',), ('roommate_profiles',)]


In [ ]:
# Adding test users
cursor.execute("""
    INSERT INTO roommate_profiles (
        firstname, lastname, case_email, gender, gender_preference, housing, year, major, major_preference,
        clean, quiet, sleep, greek_life, guests, language, cook, smoke, against_smoker, drink, against_drinker,
        pets, against_pet, politics, politics_preference, religion,
        religion_preference, bio
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    'Haley', 'Smith', 'haleysmith@case.edu', 'Female', 'Female', 'Off-Campus', 'Incoming Upperclassman',
    'Arts & Sciences', 'No Preference', 'Very Clean', 'Very Quiet', 'Before 10PM', 'Yes', '3-4 times/week', 'Spanish', 'Everyday', 'No', 'No',
    'No', 'No', 'None', 'No', 'Liberal', 'Liberal', 'Atheism/Agnosticism', 'Atheism/Agnosticism',
    'Looking for a responsible and quiet roommate.'
))

In [ ]:
cursor.execute("""
    INSERT INTO roommate_profiles (
        firstname, lastname, case_email, gender, gender_preference, housing, year, major, major_preference,
        clean, quiet, sleep, greek_life, guests, language, cook, smoke, against_smoker, drink, against_drinker,
        pets, against_pet, politics, politics_preference, religion,
        religion_preference, bio
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    'Mia', 'Cay', 'miacay@case.edu', 'Female', 'Female', 'Off-Campus', 'Incoming Upperclassman',
    'Arts & Sciences', 'No Preference', 'Somewhat Clean', 'Very Quiet', 'Before 10PM', 'Yes', '3-4 times/week', 'English', 'Everyday', 'No', 'No',
    'No', 'No', 'None', 'No', 'Liberal', 'Liberal', 'Judaism', 'No Preference',
    'Want a roommate that knows how to throw a party.'
))

In [ ]:
cursor.execute("""
    INSERT INTO roommate_profiles (
        firstname, lastname, case_email, gender, gender_preference, housing, year, major, major_preference,
        clean, quiet, sleep, greek_life, guests, language, cook, smoke, against_smoker, drink, against_drinker,
        pets, against_pet, politics, politics_preference, religion,
        religion_preference, bio
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", (
    'Joel', 'Grant', 'joelgrant@case.edu', 'Male', 'Female', 'Off-Campus', 'Incoming Upperclassman',
    'Engineering', 'No Preference', 'Somewhat Clean', 'Very Quiet', '12AM - 2AM', 'No', '3-4 times/week', 'English', 'Everyday', 'No', 'No',
    'No', 'No', 'None', 'No', 'Liberal', 'Liberal', 'Christianity', 'No Preference',
    'Just vibin.'
))

In [ ]:
cursor.execute("""INSERT INTO user_hobbies (userID, hobbyID)
VALUES
    (1, (SELECT hobbyID FROM hobbies WHERE hobby = 'Cooking')),
    (1, (SELECT hobbyID FROM hobbies WHERE hobby = 'Traveling')),
    (2, (SELECT hobbyID FROM hobbies WHERE hobby = 'Cooking')),
    (2, (SELECT hobbyID FROM hobbies WHERE hobby = 'Reading'));
""")

In [ ]:
connection.commit()

In [ ]:
cursor.execute("SELECT * FROM roommate_profiles")

In [ ]:
# Prints each row of the roommate_profiles table
for i in cursor.fetchall():
  print(i)

(1, 'Haley', 'Smith', 'haleysmith@case.edu', 'Female', 'Female', 'Off-Campus', 'Incoming Upperclassman', 'Arts & Sciences', 'No Preference', 'Very Clean', 'Very Quiet', 'Before 10PM', 'Yes', '3-4 times/week', 'Spanish', 'Everyday', 'No', 'No', 'No', 'No', 'None', 'No', 'Liberal', 'Liberal', 'Atheism/Agnosticism', 'Atheism/Agnosticism', 'Looking for a responsible and quiet roommate.')
(2, 'Mia', 'Cay', 'miacay@case.edu', 'Female', 'Female', 'Off-Campus', 'Incoming Upperclassman', 'Arts & Sciences', 'No Preference', 'Somewhat Clean', 'Very Quiet', 'Before 10PM', 'Yes', '3-4 times/week', 'English', 'Everyday', 'No', 'No', 'No', 'No', 'None', 'No', 'Liberal', 'Liberal', 'Judaism', 'No Preference', 'Want a roommate that knows how to throw a party.')


In [ ]:
cursor.execute("PRAGMA foreign_keys = ON")

In [ ]:
cursor.execute("SELECT * FROM user_hobbies")

In [ ]:
for i in cursor.fetchall():
  print(i)

(1, 4)
(1, 6)
(2, 4)
(2, 5)


In [ ]:
connection.close()

In [ ]:
files.download("/content/RoomieMatch.db")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Function to retrieve users from the database
def get_users_from_db():
    connection = sqlite3.connect("/content/RoomieMatch.db")
    cursor = connection.cursor()

    cursor.execute("SELECT * FROM roommate_profiles")

    # Get column names
    columns = [desc[0] for desc in cursor.description]

    # Convert rows to dictionaries
    users = [dict(zip(columns, row)) for row in cursor.fetchall()]

    connection.close()
    return users

In [ ]:
# Function to get users and their hobbies
def get_user_hobbies_from_db():
    """Retrieve user hobbies from the hobbies junction table and return as a dictionary."""
    query = "SELECT a.userID, b.hobby FROM user_hobbies a INNER JOIN hobbies b on a.hobbyID = b.hobbyID"
    connection = sqlite3.connect("/content/RoomieMatch.db")
    cursor = connection.cursor()

    results = cursor.execute(query)
    user_hobbies = {}

    for userID, hobby in results:
        if userID not in user_hobbies:
            user_hobbies[userID] = set()
        user_hobbies[userID].add(hobby)

    return user_hobbies

In [ ]:
get_user_hobbies_from_db()

{1: {'Cooking', 'Traveling'}, 2: {'Cooking', 'Reading'}}

In [ ]:
import itertools

# Basic matching roommate algorithm
def match_roommates(users):
    matches = []

    # Get hobbies from the junction table
    user_hobbies = get_user_hobbies_from_db()

    for user1, user2 in itertools.combinations(users, 2):
        # Must Match Criteria
        if user1['gender_preference'] != 'No Preference' and user1['gender_preference'] != user2['gender']:
            continue
        if user2['gender_preference'] != 'No Preference' and user2['gender_preference'] != user1['gender']:
            continue
        if user1['housing'] != user2['housing']:
            continue
        if user1['housing'] == 'On-Campus' and user1['year'] != user2['year']:
            continue
        if user1['against_smoker'] == 'Yes' and user2['smoke'] == 'Yes':
            continue
        if user2['against_smoker'] == 'Yes' and user1['smoke'] == 'Yes':
            continue
        if user1['against_drinker'] == 'Yes' and user2['drink'] == 'Yes':
            continue
        if user2['against_drinker'] == 'Yes' and user1['drink'] == 'Yes':
            continue
        if user1['against_pet'] == 'Yes' and user2['pets'] != 'None':
            continue
        if user2['against_pet'] == 'Yes' and user1['pets'] != 'None':
            continue

        score = 0

        # Dissimilarity scoring
        distance = 0

        # Preferences with dissimilarity scale (0-3 based on distance)
        distance += abs(['Very Messy', 'Somewhat Messy', 'Somewhat Clean', 'Very Clean'].index(user1['clean']) -
                          ['Very Messy', 'Somewhat Messy', 'Somewhat Clean', 'Very Clean'].index(user2['clean']))
        distance += abs(['Very Loud', 'Somewhat Loud', 'Somewhat Quiet', 'Very Quiet'].index(user1['quiet']) -
                          ['Very Loud', 'Somewhat Loud', 'Somewhat Quiet', 'Very Quiet'].index(user2['quiet']))
        distance +=  abs(['Before 10PM','11PM - 12AM', '12AM - 2AM','2AM or Later'].index(user1['sleep']) -
                          ['Before 10PM','11PM - 12AM', '12AM - 2AM','2AM or Later'].index(user2['sleep']))
        distance +=  abs(['Never', '1-2 times/week', '3-4 times/week', 'Everyday'].index(user1['guests']) -
                          ['Never', '1-2 times/week', '3-4 times/week', 'Everyday'].index(user2['guests']))
        distance +=  abs(['Never', '1-2 times/week', '3-4 times/week', 'Everyday'].index(user1['cook']) -
                          ['Never', '1-2 times/week', '3-4 times/week', 'Everyday'].index(user2['cook']))

        # Black-and-white preference distances (0 point if match; 1 point if no match)
        if user1['major_preference'] != 'No Preference' and user1['major_preference'] != user2['major']:
            score += 1
        if user2['major_preference'] != 'No Preference' and user2['major_preference'] != user1['major']:
            score += 1
        if user1['politics_preference'] != 'No Preference' and user1['politics_preference'] != user2['politics']:
            score += 1
        if user2['politics_preference'] != 'No Preference' and user2['politics_preference'] != user1['politics']:
            score += 1
        if user1['religion_preference'] != 'No Preference' and user1['religion_preference'] != user2['religion']:
            score += 1
        if user2['religion_preference'] != 'No Preference' and user2['religion_preference'] != user1['religion']:
            score += 1
        if user1['greek_life'] != user2['greek_life']:
            score += 1
        if user1['language'] != user2['language']:
            score += 1

        # Default hobbies score (starting at 0, add for mismatches)
        hobbies_score = 0

        # Retrieve hobbies from the dictionary
        user1_hobbies = user_hobbies.get(user1['userID'], set())
        user2_hobbies = user_hobbies.get(user2['userID'], set())

        hobbies_score = len(user1_hobbies.symmetric_difference(user2_hobbies))

        # Add hobbies dissimilarity score to total distance score
        score += distance + hobbies_score

        matches.append((user1['userID'], user2['userID'], score))

    # Sort matches by ascending score (closest users first)
    matches.sort(key=lambda x: x[2], reverse=False)
    return matches

In [ ]:
# Get users from the database
users = get_users_from_db()

# Call function with all users
matched_pairs = match_roommates(users)

# Print results (lower distance score = higher compatibility score = better match)
for match in matched_pairs:
    adjusted_score = 100 - match[2]  # Convert distance score to compatibility score
    print(f"Match User {match[0]} with User {match[1]} - Combatibility Score: {adjusted_score}")

Match: User 1 with User 2 - Combatibility Score: 95
